In [40]:
using DataStructures: DefaultDict
using Printf

In [41]:
#function to preprocessing the text
function clean_text(text)
punctuation=r"[(\(\)\",.!?;)]"
text=replace(lowercase(text),
    r"<.*?>"=>" ")
text=replace(
    text,
    punctuation=>"")
 text
end

clean_text (generic function with 1 method)

In [42]:
#read train data
postive_doc=Dict()
negative_doc=Dict()

foreach(readdir("./aclImdb/train/pos/")) do f
    postive_doc[f]=clean_text(read(string("./aclImdb/train/pos/",f),String))
end

foreach(readdir("./aclImdb/train/neg/")) do f
    negative_doc[f]=clean_text(read(string("./aclImdb/train/neg/",f),String))
end

In [43]:
#count function initialization
count_dict_pos = DefaultDict(0)
count_dict_neg=DefaultDict(0)
count_by_word_pos(x)=count_dict_pos[x]+=1
count_by_word_neg(x)=count_dict_neg[x]+=1

count_by_word_neg (generic function with 1 method)

In [44]:
#count the words in the documents
for x in split.(values(postive_doc))
    count_by_word_pos.(x)
end
for x in split.(values(negative_doc))
    count_by_word_neg.(x)
end

In [45]:
global_dict=merge(+,count_dict_pos,count_dict_neg);
global_dict=Dict(k=>global_dict[k] for k in keys(global_dict) if global_dict[k]>10);

In [46]:
w_count_dict_pos=Dict(k=>count_dict_pos[k] for k in keys(global_dict));
w_count_dict_neg=Dict(k=>count_dict_neg[k] for k in keys(global_dict));

In [47]:
unkown_keys=setdiff(collect(keys(count_dict_pos)), collect(keys(w_count_dict_pos)));
unk_pos=Dict(k=>get(count_dict_pos,k,0) for k in unkown_keys);
unk_neg=Dict(k=>get(count_dict_neg,k,0) for k in unkown_keys);

In [48]:
@show sum(values(w_count_dict_pos))+sum(values(unk_pos))
@show sum(values(w_count_dict_neg))+sum(values(unk_neg))
@show sum(values(unk_pos))
@show sum(values(unk_neg))

sum(values(w_count_dict_pos)) + sum(values(unk_pos)) = 2928246
sum(values(w_count_dict_neg)) + sum(values(unk_neg)) = 2795462
sum(values(unk_pos)) = 115764
sum(values(unk_neg)) = 50684


50684

In [49]:
#Number of classes and their frequency 
total_neg_word_number=sum(values(w_count_dict_neg))
total_pos_word_number=sum(values(w_count_dict_pos))
pos_class_ratio=length(postive_doc)/(length(postive_doc)+length(negative_doc))
neg_class_ratio=length(negative_doc)/(length(postive_doc)+length(negative_doc))

0.5

In [50]:
dict_pos_pro=Dict(k=>w_count_dict_pos[k]/total_pos_word_number for k in keys(w_count_dict_pos))
dict_pos_pro["<UNK>"]=sum(values(unk_pos))/total_pos_word_number
dict_neg_pro=Dict(k=>w_count_dict_neg[k]/total_neg_word_number for k in keys(w_count_dict_neg))
dict_neg_pro["<UNK>"]=sum(values(unk_neg))/total_neg_word_number

0.018465609969185122

In [51]:
get_prob_post_word(x)=get(dict_pos_pro,x,dict_pos_pro["<UNK>"])
get_prob_negat_word(x)=get(dict_neg_pro,x,dict_neg_pro["<UNK>"])

get_prob_post_text(x)=  log10(pos_class_ratio) + sum(log10.(get_prob_post_word.(split(clean_text(x)))))
get_prob_negat_text(x)=log10(neg_class_ratio) + sum(log10.(get_prob_negat_word.(split(clean_text(x)))))

get_prob_negat_text (generic function with 1 method)

In [52]:
TP=0.0
FN=0.0
FP=0.0
TN=0.0

0.0

In [53]:
#test accuarcy calculating
for x in readdir("./aclImdb/train/pos/")
    #println("\nObject: ", f)
     text=read(string("./aclImdb/train/pos/",x),String)
     p= get_prob_post_text(text)
     n=get_prob_negat_text(text)
    if p >=n
        TP+=1
    else
        FP+= 1
    end
end

for x in readdir("./aclImdb/test/neg/")
    text=clean_text(read(string("./aclImdb/test/neg/",x),String))
    if get_prob_negat_text(text)>=get_prob_post_text(text);TN+=1
    else;FN+=1;end
end


In [54]:
accuarcy(TP,FN,FP,TN)=(TN+TP)/(TP+FN+FP+TN)
recall(TP,FN)=TP/(TP+FN)
precesion(TP,FP)=TP/(TP+FP)

precesion (generic function with 1 method)

In [55]:
println("The accuarcy is $(@sprintf("%.2f", accuarcy(TP,FN,FP,TN)*100)) %")

The accuarcy is 82.04 %


In [56]:
"The recall is $(@sprintf("%.2f", recall(TP,FN)*100)) %"

"The recall is 77.57 %"

In [57]:
"The precesion is $(@sprintf("%.2f", precesion(TP,FP)*100)) %"

"The precesion is 90.15 %"

In [79]:
print("Total=$(TP+FN+FP+TN)\n\t\t\tActual True\tActual False\n\nTrue Predicition\t$TP\t\t$FP\n\nFalse Prediction\t$FN\t\t$TN")

Total=25000.0
			Actual True	Actual False

True Predicition	11269.0		1231.0

False Prediction	3259.0		9241.0